Utilizar o conjunto de dados Titanic.

Entregáveis:
* Conjunto de teste não utilizado no treinamento
* Análise dos resultados (porque escolheu esse modelo, como foi a avaliação, etc)
* Utilizar FastAPI para fazer o predict de cada amostra individualmente

Dicas:
* Realizar limpeza dos dados
* Realizar experimentos utilizando normalização, padronização e dados brutos
* Fazer categorização dos dados. Se fizer categorização, realizar experimentos utilizando OneHotEncoding
* Realizer experimentos selecionando atributos
* Utilizar técnicas de otimização de hiperparametros


## Importações

In [28]:
import pandas as pd
from sklearn.preprocessing import Normalizer,LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [29]:
df_test = pd.read_csv("datasets/test.csv")
df_train = pd.read_csv("datasets/train.csv")
df_test_surv = pd.read_csv('datasets/gender_submission.csv')

## Recebendo dados e analizando

In [30]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [31]:
y_test = df_test_surv["Survived"]

In [32]:
df_proc = df_train.drop(columns=["PassengerId","Name","Ticket","Fare","Cabin","Embarked",])
x_test = df_test.drop(columns=["PassengerId","Name","Ticket","Fare","Cabin","Embarked",])

In [33]:
x_train = df_proc.drop(columns=["Survived"])

In [34]:
y_train = df_proc["Survived"]

In [35]:
#Substitui os NaN's por 0
x_train = x_train.fillna(0.0)
x_test = x_test.fillna(0.0)

In [36]:
x_train["Sex"] = LabelEncoder().fit_transform(x_train["Sex"])
x_test["Sex"] = LabelEncoder().fit_transform(x_test["Sex"])

## Pré-processamento

In [37]:
transformer = Normalizer().fit(x_train)
x_train = transformer.transform(x_train)
transformer = Normalizer().fit(x_test)
x_test = transformer.transform(x_test)

## KNN

In [38]:
nknn = [1,3,5,7,9,11,13,15]
params = {'n_neighbors': nknn}
knn = KNeighborsClassifier()
gs_knn = GridSearchCV(knn,params)
gs_knn.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15]})

In [39]:
pred_knn = gs_knn.predict(x_test)

In [40]:
accuracy_score(y_test,pred_knn)

0.8325358851674641

## Decision Tree

In [53]:
criterion = ['gini','entropy']
params = {'criterion': criterion}
dt = DecisionTreeClassifier()
gs_dt = GridSearchCV(dt,params)
gs_dt.fit(x_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy']})

In [55]:
pred_dt = gs_dt.predict(x_test)

In [56]:
accuracy_score(y_test,pred_dt)

0.8325358851674641

## Random Forest

In [44]:
rfc = RandomForestClassifier()
rfc.fit(x_train,y_train)

RandomForestClassifier()

In [57]:
lista_rf = [x for x in range(50,300,50)]
params = {'n_estimators':lista_rf}
rfc = RandomForestClassifier()
clf_rf = RandomizedSearchCV(rfc, params, n_iter=3)
clf_rf.fit(x_train,y_train)

RandomizedSearchCV(estimator=RandomForestClassifier(), n_iter=3,
                   param_distributions={'n_estimators': [50, 100, 150, 200,
                                                         250]})

In [59]:
pred_rfc = clf_rf.predict(x_test)

In [60]:
accuracy_score(y_test,pred_rfc)

0.8421052631578947

## Bayes

In [47]:
gnb = GaussianNB()

In [48]:
pred = gnb.fit(x_train,y_train).predict(x_test)

In [49]:
accuracy_score(y_test,pred)

0.65311004784689

## MLP

In [77]:
clf = MLPClassifier().fit(x_train,y_train)

C:\Users\francisco.neto\.conda\envs\ai\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [78]:
pred = clf.predict(x_test)

In [79]:
accuracy_score(pred,y_test)

0.7990430622009569

### Tentando melhorar o MLP

In [80]:
lista_hl = [x for x in range(100,1001,200)]
params = {'hidden_layer_sizes':lista_hl}
#Max iter está setado em 1310, pois é a quantidade de itens
mlp = MLPClassifier(max_iter=1310)
clf_mlp = RandomizedSearchCV(mlp, params)
clf_mlp.fit(x_train,y_train)
pred2 = clf_mlp.predict(x_test)

C:\Users\francisco.neto\.conda\envs\ai\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [81]:
accuracy_score(pred2,y_test)

0.9282296650717703

## Resultados dos classificadores (com hiperparametros default)

* KNN - 0.8325358851674641
* Decision Tree - 0.8277511961722488
* Random Forrest - 0.8373205741626795
* Bayes - 0.65311004784689
* MLP - 0.7990430622009569

## Resultado dos classificadores (Utilizando GridSearch e RandomSearch)

*  KNN - 0.8325358851674641
*  Decision Tree - 0.8325358851674641
*  Random Forest - 0.8421052631578947
*  MLP - 0.930622009569378

## Conclusão

###  Baseado nos testes realizados com cinco classificadores, entendi que o mais adequado para a situação seja o Multilayer Perceptron, pois seus resultados foram de 93,06% de precisão quando utilizado técnica de otimização, como o Randomized Search e passando 1310 como a quantidade máxima de iteradores, esse 1310 se pauta pela quantidade de dados, quando executei sem alteração nos hiperparametros, encontrei o Random Forest como o melhor classificador com cerca de 83,73% de precisão.